## LightGBM test (Multi-Classification)
- player情報を付加
- 2017年の実績を付加
- 特徴量数: 97

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [8]:
ALL_MERGE = 'intermediate/all_merge_3.f'

In [9]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 112)

### 不要な列を削除

In [10]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [11]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [13]:
print(all_pitch.shape)
all_pitch.head()

(778767, 97)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,date_from_opening,elapsed_min,home,point_diff,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,curve_pit,slider_pit,shoot_pit,fork_pit,changeup_pit,sinker_pit,cutball_pit,total_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0.000000,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,196,100,NaN,24000,0.459848,0.493939,0.199242,0.023485,0.061364,0.0,0.0,2954.0,1,0,0,0,29,NaN,0,176,74,5.0,15000,0,1,1,0,32,8.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,9000,NaN,-3,19000,NaN,1,10000,4.0,4
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0.200000,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,196,100,NaN,24000,0.459848,0.493939,0.199242,0.023485,0.061364,0.0,0.0,2954.0,1,0,0,0,29,NaN,0,176,74,5.0,15000,0,1,1,0,32,8.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,9000,NaN,-3,19000,NaN,1,10000,4.0,4
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0.383333,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,196,100,NaN,24000,0.459848,0.493939,0.199242,0.023485,0.061364,0.0,0.0,2954.0,1,0,0,0,29,NaN,0,176,74,5.0,15000,0,1,1,0,32,8.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,9000,NaN,-3,19000,NaN,1,10000,4.0,4
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.016667,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,196,100,NaN,24000,0.459848,0.493939,0.199242,0.023485,0.061364,0.0,0.0,2954.0,1,0,0,0,29,NaN,0,180,74,1.0,7040,0,0,0,1,40,22.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,16960,NaN,-11,19000,NaN,1,2040,18.0,12
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,1.200000,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,196,100,NaN,24000,0.459848,0.493939,0.199242,0.023485,0.061364,0.0,0.0,2954.0,1,0,0,0,29,NaN,0,180,74,1.0,7040,0,0,0,1,40,22.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,16960,NaN,-11,19000,NaN,1,2040,18.0,12


### train
- 行数: 257117

In [14]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 97)

### test
- 行数: 521650

In [15]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 97)

In [16]:
del all_pitch
gc.collect()

64

In [17]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [18]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [19]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8

In [20]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'num_leaves' : 31,
        'seed' : 0,
        'learning_rate' : 0.1,
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=8000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
print('Best CV score:', cv_results['multi_logloss-mean'][-1])

[100]	cv_agg's multi_logloss: 1.1322 + 0.00171313
[200]	cv_agg's multi_logloss: 1.12277 + 0.0025523
[300]	cv_agg's multi_logloss: 1.11903 + 0.00271275
[400]	cv_agg's multi_logloss: 1.11747 + 0.0029211
[500]	cv_agg's multi_logloss: 1.11733 + 0.0030824
Best num_boost_round: 453
Best CV score: 1.1172290313076036
CPU times: user 43min 44s, sys: 37.1 s, total: 44min 21s
Wall time: 11min 49s


In [21]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [22]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 11min 54s, sys: 3.74 s, total: 11min 57s
Wall time: 3min 3s


### Feature Importance

In [23]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(10)

,feat_name,feat_imp
1,pitch_cnt_in_game,2872
85,salary_dif_p-b,2914
91,salary_dif_b-c,2975
65,weight_bat,3107
4,pitch_cnt_in_bat,3263
0,No_in_game,3344
25,elapsed_min,3768
8,pitcher_cnt_in_inning,4684
7,pitcher_cnt_in_game,5741
24,date_from_opening,6252


In [24]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 29s, sys: 1.85 s, total: 6min 31s
Wall time: 1min 42s


In [25]:
predict

array([[3.45377277e-01, 3.60800461e-02, 1.40133749e-01, ...,
        1.60715581e-06, 9.17215039e-07, 2.36544757e-01],
       [4.40118297e-01, 1.02386070e-01, 1.41176070e-01, ...,
        2.17058384e-06, 5.40187229e-07, 6.62313147e-02],
       [3.57414506e-01, 1.18587844e-01, 1.57384427e-01, ...,
        3.07489710e-06, 3.21798651e-06, 1.39297172e-01],
       ...,
       [4.49615218e-01, 8.98997119e-02, 8.35874757e-04, ...,
        1.13962079e-05, 1.54540549e-06, 3.67111681e-04],
       [5.04004468e-01, 3.16694475e-02, 5.99354545e-04, ...,
        4.10505747e-06, 7.67023493e-07, 2.12536631e-04],
       [4.86009103e-01, 3.87635917e-02, 7.51941478e-04, ...,
        5.63312205e-06, 1.16675622e-06, 2.71495963e-04]])

## 結果出力

In [26]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.345377,0.036080,0.140134,0.240034,0.001828,0.000002,9.172150e-07,0.236545
1,1,0.440118,0.102386,0.141176,0.247371,0.002715,0.000002,5.401872e-07,0.066231
2,2,0.357415,0.118588,0.157384,0.153983,0.073327,0.000003,3.217987e-06,0.139297
3,3,0.319470,0.107775,0.220075,0.192740,0.018629,0.000003,2.354312e-06,0.141306
4,4,0.216925,0.045728,0.315326,0.220882,0.018295,0.000003,2.227534e-06,0.182838


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [27]:
submit.mean()

index    260824.500000
0             0.486507
1             0.074009
2             0.188220
3             0.059655
4             0.078583
5             0.051244
6             0.009389
7             0.052393
dtype: float64

In [28]:
submit.to_csv('submit/lgbm_3.csv', header=False, index=False)

### 評価結果
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806